Repaso Extracción de Datos (Beautiful Soup, Selenium y API's)

In [3]:
!pip install tqdm
!pip install python-dotenv

In [4]:
import pandas as pd

from time import sleep
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup

1. Consulta de Datos de Libros (API Books API - Google Books):

- Utiliza la API Books API para buscar libros de Stephen King. Aquí el link a su documentación.

- Realiza una solicitud a la API y almacena los datos de los libros de Stephen King’s.

In [5]:
api_key = "89LqFqaMKAxlwBTdnAhXKAGgrYGhRXTS"

In [6]:
url = f"https://api.nytimes.com/svc/books/v3/reviews.json?author=Stephen%20King&api-key={api_key}"

llamada = requests.get(url)

print(f'La respuesta a la llamada es {llamada.status_code}')

respuesta_json = llamada.json()

La respuesta a la llamada es 200


In [7]:
respuesta_json.keys()

dict_keys(['status', 'copyright', 'num_results', 'results'])

In [8]:
respuesta_json['copyright']

'Copyright (c) 2024 The New York Times Company.  All Rights Reserved.'

In [9]:
respuesta_json['num_results']

66

In [10]:
respuesta_json['results'][0]

{'url': 'http://www.nytimes.com/2011/11/13/books/review/11-22-63-by-stephen-king-book-review.html',
 'publication_dt': '2011-11-13',
 'byline': 'ERROL MORRIS',
 'book_title': '11/22/63',
 'book_author': 'Stephen King',
 'summary': 'Stephen King’s time traveler tries to undo some painful history.',
 'uuid': '00000000-0000-0000-0000-000000000000',
 'uri': 'nyt://book/00000000-0000-0000-0000-000000000000',
 'isbn13': ['9780307951434',
  '9780606351461',
  '9781442344280',
  '9781442344303',
  '9781442391635',
  '9781444727326',
  '9781451627282',
  '9781451627299',
  '9781451627305',
  '9781451651645',
  '9781501120602',
  '9781594135590']}

In [12]:
df_libros = pd.DataFrame(respuesta_json['results'])

df_libros.head()

,url,publication_dt,byline,book_title,book_author,summary,uuid,uri,isbn13
0,http://www.nytimes.com/2011/11/13/books/review...,2011-11-13,ERROL MORRIS,11/22/63,Stephen King,Stephen King’s time traveler tries to undo som...,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,"[9780307951434, 9780606351461, 9781442344280, ..."
1,http://www.nytimes.com/2011/10/31/books/stephe...,2011-10-31,JANET MASLIN,11/22/63,Stephen King,"Stephen King’s latest novel, “11/22/63,” tells...",00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,"[9780307951434, 9780606351461, 9781442344280, ..."
2,http://www.nytimes.com/2004/01/04/books/the-qu...,2004-01-04,ANDREW O'HEHIR,Wolves of the Calla,Stephen King,,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9781848941137]
3,http://www.nytimes.com/1993/10/24/books/in-sho...,1993-10-24,RICHARD E. NICHOLLS,Nightmares and Dreamscapes,Stephen King,,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9781441615299]
4,http://www.nytimes.com/2001/11/04/books/books-...,2001-11-04,MARY ELIZABETH WILLIAMS,Black House,Stephen King,,00000000-0000-0000-0000-000000000000,nyt://book/00000000-0000-0000-0000-000000000000,[9780375504396]


In [13]:
lista_libros=df_libros['book_title'].unique()

lista_libros

array(['11/22/63', 'Wolves of the Calla', 'Nightmares and Dreamscapes',
       'Black House', 'The Stand', 'Four Past Midnight', 'Needful Things',
       "Gerald's Game", "Everything's Eventual : 14 Dark Tales", 'It',
       'Full Dark, No Stars', 'Bag of Bones', 'Rose Madder',
       'Doctor Sleep', 'Joyland', 'Under the Dome', 'Revival',
       "Lisey's Story", 'Just After Sunset', 'Duma Key', 'Mr. Mercedes',
       "Stephen King's Danse Macabre", 'Cujo', 'Different Seasons',
       'Christine', 'Pet Sematary', 'Skeleton Crew',
       'The Eyes of the Dragon', 'Misery', 'The Tommyknockers',
       'The Dark Half', 'The Waste Lands: The Dark Tower Book III',
       'Needful Things: The Last Castle Rock Story',
       'Insomnia,Stephen King Library Ed.',
       'The Girl Who Loved Tom Gordon : A Novel',
       'On Writing:  a Memoir of the Craft', 'Dreamcatcher',
       'From a Buick 8', 'The Dark Tower VI', 'Cell', 'Finders Keepers',
       'End of Watch', 'The Outsider', 'Elevation',

2. Scraping de Datos de Libros (Selenium - Book Center):

- Utiliza la biblioteca Selenium para automatizar la navegación en la página web de Book Center, una tienda en línea de libros.

- Realiza búsquedas de libros en el sitio web y navega por las páginas de resultados.

- Extrae información detallada de los libros, como el precio (con y sin descuento si está disponible), editorial, año de edición y número de páginas.

- Almacena estos datos en una estructura de datos, como una lista de diccionarios.

In [14]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.bookcenter.es")

sleep(2)

driver.find_element("css selector", "#cookie-accept").click()

sleep(2)

lista_urls = []

for i in tqdm(lista_libros):
    try:
        driver.find_element('css selector', '#busqueda > input.search-text').send_keys(i, Keys.ENTER)
        sleep(3)
        url_libro = driver.find_element('css selector', '#main-container > div > div > div.row > ul > li:nth-child(1) > form > dl > dd.title > a').get_attribute("href")
        lista_urls.append(url_libro)
    except:
        pass
    
    sleep(3)
    
driver.close()

100%|██████████| 45/45 [04:48<00:00,  6.42s/it]


In [15]:
lista_urls

['https://www.bookcenter.es/libro/the-dark-tower-5-the-wolves-of-calla_24433',
 'https://www.bookcenter.es/libro/los-hombres-lobo_97210',
 'https://www.bookcenter.es/libro/apocalipsis-el-libro-en-el-que-se-basa-la-serie-the-stand_156879',
 'https://www.bookcenter.es/libro/co8-geralds-game_34975',
 'https://www.bookcenter.es/libro/full-dark-no-stars_99349',
 'https://www.bookcenter.es/libro/el-retrato-de-rose-madder_29235',
 'https://www.bookcenter.es/libro/doctor-sleep_95001',
 'https://www.bookcenter.es/libro/joyland_112137',
 'https://www.bookcenter.es/libro/under-the-dome_54241',
 'https://www.bookcenter.es/libro/revival_116052',
 'https://www.bookcenter.es/libro/co6-lisey-s-story-td_29076',
 'https://www.bookcenter.es/libro/duma-key_50324',
 'https://www.bookcenter.es/libro/10-000-millas-para-encontrarte-bali-2_180292',
 'https://www.bookcenter.es/libro/cujo_2884',
 'https://www.bookcenter.es/libro/christine_1860',
 'https://www.bookcenter.es/libro/skeleton-crew_34714',
 'https://w

3. Creación de una Base de Datos Local:

- Diseña una BBDD para almacenar la información de los libros.3. Creación de una Base de Datos Local:

- Diseña una BBDD para almacenar la información de los libros.

In [16]:
diccionario_libros = {'Título':[],
                            'Precio':[],
                            'Editorial':[],
                            'Año_edición':[],
                            'Número_páginas':[]}

In [17]:
for i in lista_urls:
    
    llamada=requests.get(i)
    sopa=BeautifulSoup(llamada.content, 'html.parser')
    detalles_sopa = sopa.find_all("dl", {"class": "summary col-md-8 col-sm-8"})[0].text.split("\n") #este es selector con toda la info del libro (editorial, precio, etc)
    
    diccionario_libros['Título'].append(detalles_sopa[1])

    if len(detalles_sopa[7])==0:
        diccionario_libros['Precio'].append(detalles_sopa[8])
    else:
        diccionario_libros['Precio'].append(detalles_sopa[7])

    for indice, elemento in enumerate(detalles_sopa):
        if elemento == 'Editorial:':
            diccionario_libros['Editorial'].append(detalles_sopa[indice+1])
        
        elif elemento == 'Páginas:':
            diccionario_libros['Número_páginas'].append(detalles_sopa[indice+1])

        elif elemento == 'Año de edición: ':
            diccionario_libros['Año_edición'].append(detalles_sopa[indice+1])

    if len(diccionario_libros["Editorial"]) != len(diccionario_libros["Año_edición"]):
        diccionario_libros["Año_edición"].append("desconocido")

    if len(diccionario_libros["Editorial"]) != len(diccionario_libros["Número_páginas"]):
        diccionario_libros["Número_páginas"].append("desconocido")

diccionario_libros

{'Título': ['THE DARK TOWER 5. THE WOLVES OF CALLA',
  'LOS HOMBRES-LOBO',
  'APOCALIPSIS. EL LIBRO EN EL QUE SE BASA LA SERIE THE STAND',
  'CO8 GERALDS GAME',
  'FULL DARK NO STARS',
  'EL RETRATO DE ROSE MADDER',
  'DOCTOR SLEEP',
  'JOYLAND',
  'UNDER THE DOME',
  'REVIVAL',
  'CO6 LISEY´S STORY TD',
  'DUMA KEY',
  '10.000 MILLAS PARA ENCONTRARTE (BALI 2)',
  'CUJO',
  'CHRISTINE',
  'SKELETON CREW',
  'MISERY',
  'EL CAZADOR DE SUEÑOS',
  'SONG OF SUSANNAH THE DARK TOWER',
  'EL JUEGO DE ANDRÓMEDA',
  'FINDERS KEEPERS',
  'END OF WATCH',
  'THE OUTSIDER',
  'ELEVATION',
  'HISTORIA DE LA MODA. DEL SIGLO XVIII AL SIGLO XX'],
 'Precio': ['9,50 €',
  '3,00 €',
  '12,95 €',
  '9,95 €',
  '10,99 €',
  '10,95 €',
  '10,95 €',
  '12,95 €',
  '12,95 €',
  '9,95 €',
  '22,00 €',
  '13,95 €',
  '18,95 €',
  '11,95 €',
  '11,95 €',
  '9,95 €',
  '16,95 €',
  '12,90 €',
  '24,00 €',
  '16,90 €',
  '9,95 €',
  '9,95 €',
  '12,95 €',
  '10,95 €',
  '60,00 €'],
 'Editorial': ['LIBROS EN INGLES'

In [21]:
df_libros = pd.DataFrame(diccionario_libros)

df_libros

,Título,Precio,Editorial,Año_edición,Número_páginas
0,THE DARK TOWER 5. THE WOLVES OF CALLA,"9,50 €",LIBROS EN INGLES,2006,616
1,LOS HOMBRES-LOBO,"3,00 €",SIRUELA OFERTA,2002,288
2,APOCALIPSIS. EL LIBRO EN EL QUE SE BASA LA SER...,"12,95 €",DEBOLSILLO,2021,1584
3,CO8 GERALDS GAME,"9,95 €",LIBROS EN INGLES,desconocido,desconocido
4,FULL DARK NO STARS,"10,99 €",LIBROS EN INGLES,2013,desconocido
5,EL RETRATO DE ROSE MADDER,"10,95 €",DEBOLSILLO,2018,496
6,DOCTOR SLEEP,"10,95 €",LIBROS EN INGLES,2014,499
7,JOYLAND,"12,95 €",LIBROS EN FRANCES,2015,400
8,UNDER THE DOME,"12,95 €",LIBROS EN INGLES,2010,875
9,REVIVAL,"9,95 €",DEBOLSILLO,2016,416


In [22]:
df_libros.to_csv('libros_stephen_2.csv')